In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs               
import cartopy.feature as cfeature         
import cartopy.util as cutil
import xarray as xr                        
import numpy as np
import pandas as pd
import xesmf as xe

In [2]:
ozone_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.O3.200201-202412.nc")
pdeldry_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PDELDRY.200201-202412.nc")
ps_dataset = xr.open_dataset('/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PS.200201-202412.nc')

In [3]:
ozone = ozone_dataset["O3"]

In [4]:
p0 = ozone_dataset["P0"]
hyai = ozone_dataset["hyai"]
hybi = ozone_dataset["hybi"]
ps = ps_dataset['PS']
pdeldry = pdeldry_dataset['PDELDRY']
lev = ozone_dataset.coords['lev']
num_lev = lev.shape[0]

# convert to hPa from Pa
p0 = p0.copy() / 100
ps = ps.copy() / 100
pdeldry = pdeldry.copy() / 100 


Create climatology

In [5]:
start_date = '2005-02-01'
end_date = '2025-01-01'

# group the 240 month dates based on calendar months for both PDELDRY and O3 variables

truncated_pdeldry = pdeldry.sel(time=slice(start_date, end_date))
pdeldry_monthly_mean = truncated_pdeldry.groupby('time.month').mean('time')
pdeldry_monthly_mean = pdeldry_monthly_mean.transpose('lev','month','lat','lon')

truncated_ozone = ozone.sel(time=slice(start_date, end_date))
ozone_monthly_mean = truncated_ozone.groupby('time.month').mean('time')
ozone_monthly_mean = ozone_monthly_mean.transpose('lev','month','lat','lon')

truncated_ps = ps.sel(time=slice(start_date, end_date))
ps_monthly_mean = truncated_ps.groupby('time.month').mean('time')
ps_monthly_mean = ps_monthly_mean.transpose('month','lat','lon')

Tropospheric column calculations

Calculating pressure at hybrid levels

p(k) = a(k) * p0 + b(k) * ps

In [6]:
# constants / conversion factor
NAv = 6.0221415e+23                       # molecules in mole
g = 9.81                                  # gravity
MWair = 28.94                             # g/mol
xp_const = (NAv * 10)/(MWair*g)           # scaling factor, pa to hPa and cm to m
DU_CONVERSION = 2.69 * 10**16

In [7]:
# Initialize pressure edge arrays
mod_press_edge_top = xr.zeros_like(ozone_monthly_mean)
mod_press_edge_bottom = xr.zeros_like(ozone_monthly_mean)

In [8]:
# Calculate pressure edge arrays
# CAM-chem layer indices start at the top and end at the bottom
for i in range(num_lev):
    mod_press_edge_top[i,:,:,:] = hyai[i]*p0 + hybi[i]*ps_monthly_mean
    mod_press_edge_bottom[i,:,:,:] = hyai[i+1]*p0 + hybi[i+1]*ps_monthly_mean

In [9]:
#print(mod_press_edge_bottom.sel(lat=slice(40,40.5),lon=slice(150,150.5), month=1).values)

In [10]:
test = mod_press_edge_top - 300
test

<xarray.DataArray 'O3' (lev: 32, month: 12, lat: 192, lon: 288)> Size: 85MB
array([[[[-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
         [-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
         [-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
         ...,
         [-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
         [-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
         [-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475]],

        [[-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
         [-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
         [-297.74475, -297.74475, -297.74475, ..., -297.74475,
          -297.74475, -297.74475],
...
         [ 698.30316,  698.2886 ,  698.27405, ...,  698.3484 ,
           698.33307,  698.31793],
         [ 698.5645 ,  698.5557 ,  698.5472 , ...,  698.591  ,
           698.5822 ,  698.57336],
         [ 698.7988 ,  698.7988 ,  698.7988 , ...,  698.7988 ,
           698.7988 ,  698.7988 ]],

        [[ 368.75574,  368.75574,  368.75574, ...,  368.75574,
           368.75574,  368.75574],
         [ 380.42743,  380.14526,  379.83032, ...,  381.27545,
           380.9959 ,  380.70294],
         [ 388.25195,  387.65247,  387.03607, ...,  390.02106,
           389.43652,  388.8438 ],
         ...,
         [ 696.4405 ,  696.4331 ,  696.426  , ...,  696.4647 ,
           696.4563 ,  696.44824],
         [ 696.7476 ,  696.74304,  696.7387 , ...,  696.7616 ,
           696.7569 ,  696.7522 ],
         [ 697.04016,  697.04016,  697.04016, ...,  697.04016,
           697.04016,  697.04016]]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lev      (lev) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [11]:
filtered_300hpa_upper = mod_press_edge_top.where(mod_press_edge_top >= 300, drop=False)
filtered_300hpa_lower = mod_press_edge_bottom.where(mod_press_edge_bottom >= 300, drop=False)

In [12]:
filtered_deltap = filtered_300hpa_lower-filtered_300hpa_upper
filtered_deltap = filtered_deltap.fillna(0)

In [13]:
filtered_deltap[19][1]

<xarray.DataArray 'O3' (lat: 192, lon: 288)> Size: 221kB
array([[44.26538 , 44.26538 , 44.26538 , ..., 44.26538 , 44.26538 ,
        44.26538 ],
       [45.280853, 45.255188, 45.226654, ..., 45.358063, 45.33261 ,
        45.30597 ],
       [45.968018, 45.91391 , 45.858368, ..., 46.127777, 46.07498 ,
        46.021454],
       ...,
       [73.43512 , 73.435394, 73.43573 , ..., 73.43475 , 73.434814,
        73.43494 ],
       [73.49802 , 73.49808 , 73.49814 , ..., 73.49808 , 73.49805 ,
        73.49802 ],
       [73.563416, 73.563416, 73.563416, ..., 73.563416, 73.563416,
        73.563416]], dtype=float32)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lev      float64 8B 446.0
    month    int64 8B 2

In [14]:
test_ozone = xr.zeros_like(ozone_monthly_mean)
test_ozone = ozone_monthly_mean.where(filtered_deltap > 0)

In [15]:
test_ozone = test_ozone.fillna(0)

In [16]:
ozone_column = xr.dot(filtered_deltap, xp_const*test_ozone, dims='lev')

In [17]:
ozone_du_column = ozone_column.copy() / DU_CONVERSION

In [18]:
ozone_du_column[0].mean(dim={'lat','lon'}, skipna=True)

<xarray.DataArray 'O3' ()> Size: 4B
array(18.407604, dtype=float32)
Coordinates:
    month    int64 8B 1

In [19]:
ozone_du_column

<xarray.DataArray 'O3' (month: 12, lat: 192, lon: 288)> Size: 3MB
array([[[ 8.793458 ,  8.793458 ,  8.793458 , ...,  8.793458 ,
          8.793458 ,  8.793458 ],
        [ 8.913348 ,  8.910356 ,  8.906912 , ...,  8.922044 ,
          8.9192   ,  8.9162035],
        [ 9.003218 ,  8.997766 ,  8.992071 , ...,  9.019067 ,
          9.013924 ,  9.00858  ],
        ...,
        [25.044416 , 25.043062 , 25.04158  , ..., 25.048979 ,
         25.047306 , 25.045801 ],
        [25.079252 , 25.077932 , 25.076605 , ..., 25.083664 ,
         25.08213  , 25.080652 ],
        [25.170557 , 25.170557 , 25.170559 , ..., 25.170557 ,
         25.170557 , 25.170557 ]],

       [[ 8.859958 ,  8.859958 ,  8.859958 , ...,  8.859958 ,
          8.859958 ,  8.859958 ],
        [ 8.960573 ,  8.958755 ,  8.956383 , ...,  8.966861 ,
          8.964668 ,  8.962439 ],
        [ 9.004455 ,  9.001396 ,  8.998057 , ...,  9.013862 ,
          9.010644 ,  9.007515 ],
...
        [24.469759 , 24.469044 , 24.468422 , ..., 24.471931 ,
         24.471243 , 24.470531 ],
        [24.515839 , 24.514666 , 24.513483 , ..., 24.519415 ,
         24.518211 , 24.516996 ],
        [24.532156 , 24.532156 , 24.532156 , ..., 24.532156 ,
         24.532152 , 24.532156 ]],

       [[ 8.773393 ,  8.773393 ,  8.773393 , ...,  8.773393 ,
          8.773393 ,  8.773393 ],
        [ 8.953869 ,  8.951372 ,  8.948238 , ...,  8.961164 ,
          8.958796 ,  8.95622  ],
        [ 9.094081 ,  9.088509 ,  9.0826845, ...,  9.109967 ,
          9.104846 ,  9.099496 ],
        ...,
        [24.650265 , 24.648485 , 24.646666 , ..., 24.655785 ,
         24.653852 , 24.651993 ],
        [24.689798 , 24.688253 , 24.68672  , ..., 24.694403 ,
         24.692875 , 24.691319 ],
        [24.710001 , 24.710001 , 24.710001 , ..., 24.710003 ,
         24.710003 , 24.710001 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [20]:
filtered_deltap.sel(lat=slice(40,40.5),lon=slice(150,150.5), month=1)

<xarray.DataArray 'O3' (lev: 32, lat: 1, lon: 1)> Size: 128B
array([[[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

       [[ 0.      ]],

...

       [[79.315674]],

       [[66.93213 ]],

       [[49.744873]],

       [[28.79895 ]],

       [[27.018433]],

       [[25.019226]],

       [[22.813843]],

       [[20.415833]],

       [[17.844177]],

       [[15.074707]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 8B 40.05
  * lon      (lon) float64 8B 150.0
  * lev      (lev) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
    month    int64 8B 1

In [21]:
pdeldry_monthly_mean.sel(lat=slice(40,40.5),lon=slice(150,150.5), month=1)

<xarray.DataArray 'PDELDRY' (lev: 32, lat: 1, lon: 1)> Size: 128B
array([[[ 2.776441 ]],

       [[ 5.1262355]],

       [[ 8.397338 ]],

       [[11.179318 ]],

       [[ 9.538293 ]],

       [[ 7.8414717]],

       [[ 9.12597  ]],

       [[10.559963 ]],

       [[13.900875 ]],

       [[14.239577 ]],

...

       [[79.2244   ]],

       [[66.80873  ]],

       [[49.615974 ]],

       [[28.709347 ]],

       [[26.925869 ]],

       [[24.92831  ]],

       [[22.727964 ]],

       [[20.337385 ]],

       [[17.77446  ]],

       [[15.014552 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 8B 40.05
  * lon      (lon) float64 8B 150.0
  * lev      (lev) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
    month    int64 8B 1

In [22]:
filtered_deltap.sel(lat=slice(40,40.5),lon=slice(150,150.5), month=1)-pdeldry_monthly_mean.sel(lat=slice(40,40.5),lon=slice(150,150.5), month=1)

<xarray.DataArray (lev: 32, lat: 1, lon: 1)> Size: 128B
array([[[-2.7764411e+00]],

       [[-5.1262355e+00]],

       [[-8.3973379e+00]],

       [[-1.1179318e+01]],

       [[-9.5382929e+00]],

       [[-7.8414717e+00]],

       [[-9.1259699e+00]],

       [[-1.0559963e+01]],

       [[-1.3900875e+01]],

       [[-1.4239577e+01]],

...

       [[ 9.1270447e-02]],

       [[ 1.2339783e-01]],

       [[ 1.2889862e-01]],

       [[ 8.9603424e-02]],

       [[ 9.2563629e-02]],

       [[ 9.0915680e-02]],

       [[ 8.5878372e-02]],

       [[ 7.8447342e-02]],

       [[ 6.9717407e-02]],

       [[ 6.0154915e-02]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 8B 40.05
  * lon      (lon) float64 8B 150.0
  * lev      (lev) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
    month    int64 8B 1

Testing the estimated DU column. 

In [23]:
test_truncated_pdeldry = pdeldry.isel({pdeldry.dims[1]: slice(17, 31)})
test_truncated_ozone = ozone.isel({ozone.dims[1]: slice(17, 31)})

# group the 240 month dates based on calendar months for both PDELDRY and O3 variables

test_truncated_pdeldry = test_truncated_pdeldry.sel(time=slice(start_date, end_date))
test_pdeldry_monthly_mean = test_truncated_pdeldry.groupby('time.month').mean('time')
test_pdeldry_monthly_mean = test_pdeldry_monthly_mean.transpose('lev','month','lat','lon')

test_truncated_ozone = test_truncated_ozone.sel(time=slice(start_date, end_date))
test_ozone_monthly_mean = test_truncated_ozone.groupby('time.month').mean('time')
test_ozone_monthly_mean = test_ozone_monthly_mean.transpose('lev','month','lat','lon')

In [24]:
test_column = xr.dot(test_pdeldry_monthly_mean, xp_const*test_ozone_monthly_mean, dims='lev')

In [25]:
ozone_du_column = test_column.copy() / DU_CONVERSION

In [26]:
ozone_du_column[0].mean(dim={'lat','lon'}, skipna=True)

<xarray.DataArray ()> Size: 4B
array(20.800097, dtype=float32)
Coordinates:
    month    int64 8B 1